In [15]:
import boto3
from sagemaker import Session
from sagemaker import get_execution_role

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [16]:
# Load the scaler
import s3fs
import os
import joblib

fs = s3fs.S3FileSystem()
output_file = "s3://eliezerraj-908671954593-dataset/customer/scaler.joblib"

with fs.open(output_file, 'rb') as f:
    scaler_load = joblib.load(f)

print("model loaded !")

model loaded !


In [17]:
import numpy as np

customer_01 = np.array([[50.0, 1.0, 3.0, 4.0]])

print("customer_01: ", customer_01)

df_payload_scaled = scaler_load.fit_transform(customer_01).astype('float32')
df_payload_scaled

customer_01:  [[50.  1.  3.  4.]]


array([[0., 0., 0., 0.]], dtype=float32)

In [9]:
import numpy as np
# Endpoint invocation
endpoint_name="kmeans-2024-04-29-14-58-01-252"
payload = b"0.3335704,2.0431986,0.8342485,-0.736292"

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)

y_predict = response["Body"].read().decode()
print(y_predict)

{"predictions": [{"closest_cluster": 2.0, "distance_to_cluster": 1.6833887100219727}]}


In [12]:
import pandas as pd

df_new_customer_data = [{ 'Customer_Age': 20, 
                         'Dependent_count': 1, 
                         'Education_Level_Quality': 1, 
                         'Income_Category_Quality': 1 }]

df_new_customer_data = pd.DataFrame(df_new_customer_data)
df_new_customer_data

,Customer_Age,Dependent_count,Education_Level_Quality,Income_Category_Quality
0,20,1,1,1


In [13]:
df_data_scaled = df_new_customer_data.to_numpy().astype('float32')
df_data_scaled

array([[20.,  1.,  1.,  1.]], dtype=float32)

In [14]:
df_payload_scaled = scaler_load.fit_transform(df_data_scaled).astype('float32')
df_payload_scaled

array([[0., 0., 0., 0.]], dtype=float32)